In [ ]:
pip install transformers --upgrade


In [ ]:
pip install sentencepiece --upgrade


In [ ]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
df_str_rel = pd.read_csv('C:\\Users\\ADMIN\\Downloads\\mar_train.csv')
df_str_rel.head()

In [ ]:
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Specify the model name or path
model_name_or_path = "ai4bharat/indic-bert"

# Load the IndicBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path)

# Function to encode text and get embeddings
def encode_text(text):
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Calculate cosine similarity between two sentence embeddings
def calculate_similarity(sentence1, sentence2):
    embeddings1 = encode_text(sentence1)
    embeddings2 = encode_text(sentence2)
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]

sentence1 = "This is the first sentence."
sentence2 = "This is the second sentence."

similarity = calculate_similarity(sentence1, sentence2)
print(f"Cosine Similarity: {similarity:.4f}")


In [ ]:
true_scores = df_str_rel['Score'].values
pred_scores = []

for index,row in df_str_rel.iterrows():
  s1,s2 = row["Text"].split("\n")

  # Overlap score
  pred_scores.append(calculate_similarity(s1,s2))

In [ ]:
print("Spearman Correlation:", round(spearmanr(true_scores,pred_scores)[0],2))

In [ ]:
pip install protobuf